In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from config import Config
from pprint import pprint, pformat
from logger import model_logger
log = model_logger.getLogger('main')
log.setLevel(Config.Log.MODEL.level)

2018-01-30 21:00:13,611:root:INFO   :           getLogger:: creating logger for main under MODEL


In [4]:
%autoreload 2
from trainer import Trainer, Feeder, Predictor
from datafeed import DataFeed
from torch import nn, optim
from torch.nn import functional as F
from torch.autograd import Variable
import torch

2018-01-30 21:00:14,779:root:INFO   :           getLogger:: creating logger for main under TRAINER
2018-01-30 21:00:15,000:root:INFO   :           getLogger:: creating logger for main under DATAFEED


In [42]:
import csv
train_dataset = csv.reader(open('dataset/train.csv'))
test_dataset = csv.reader(open('dataset/test.csv'))

In [43]:
from collections import namedtuple
Sample = namedtuple('Sample', ['id','comment_text',
                               'toxic','severe_toxic','obscene',
                               'threat','insult','identity_hate'])

### Unicode to ascii text

In [44]:
import unicodedata
train_datapoints = []
for i in list(train_dataset)[1:]:
    _id, c, t, st, o, t, ins, ih = i
    t, st, o, t, ins, ih = (int(_) for _ in [t, st, o, t, ins, ih])
    c = unicodedata.normalize('NFD', c).encode('ascii','ignore').decode()
    train_datapoints.append(Sample(_id, c.lower(), t, st, o, t, ins, ih))

test_datapoints = []
for i in list(test_dataset)[1:]:
    _id, c = i
    c = unicodedata.normalize('NFD', c).encode('ascii','ignore').decode()
    test_datapoints.append(Sample(_id, c, 0, 0, 0, 0, 0, 0))

len(train_datapoints), len(test_datapoints)

(159571, 153164)

In [45]:
cond = lambda x: sum(x[2:]) >= 1
classified_train_datapoints = [p for p in train_datapoints if cond(p)]

In [46]:
len(classified_train_datapoints)

10559

In [11]:
test_datapoints[1000:1010]

[Sample(id='01ac9982edae9977', comment_text='" \n\n Dear ,Welcome to Wikipedia!Unfortunately, using your e-mail address as your username is not a good idea. Wikipedia content is extensively copied and the site itself is one of the most visited sites in the world. Any edit you make on Wikipedia will have your username attached to it, and using your email address will make you a tempting target for spammers. We recommend that you change your username at Wikipedia:Changing username in order to prevent abuse.If you need any help, simply contact me on my talk page, or go to Wikipedia:Help desk. Another option is to place  on your own talk page, and someone will come shortly to help. Remember to sign your posts on talk pages with four tildes (~~~~). Again, welcome! -  at "', toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0),
 Sample(id='01ac9ea1271d7409', comment_text='Asshole, your dirty MF people tries to steal our glory and heavy persian (tajik) history by claiming 

## Build vocabulary

#### buils INPUT_VOCAB

In [47]:
%timeit
from nltk import word_tokenize
from tqdm import tqdm
from collections import defaultdict
datapoints = train_datapoints
WORD_FREQ = defaultdict(int)
INPUT_VOCAB = [word for dp in tqdm(datapoints) for word in word_tokenize(dp.comment_text)]
OUTPUT_VOCAB = ['toxic','severe_toxic','obscene', 'threat','insult','identity_hate']

100%|██████████| 159571/159571 [01:22<00:00, 1930.68it/s]


In [48]:
INPUT_VOCAB = INPUT_VOCAB + OUTPUT_VOCAB
for word in INPUT_VOCAB: WORD_FREQ[word] += 1

WORD_FREQ_PAIRS = sorted(WORD_FREQ.items(), key=lambda x: -x[1])
print(WORD_FREQ_PAIRS[1110:1120])

INPUT_VOCAB = [x[0] for x in WORD_FREQ_PAIRS]
INPUT_VOCAB = list(set(INPUT_VOCAB))
len(INPUT_VOCAB)

[('weeks', 949), ('study', 948), ('stand', 948), ('introduction', 947), ('difficult', 946), ('controversial', 946), ('cock', 944), ('reasonable', 943), ('vandal', 942), ('local', 941)]


255050

In [49]:
print(WORD_FREQ_PAIRS[-10000:])


[('inm', 1), ('gait', 1), ('ratsassmachineelf1735', 1), ('m1a', 1), ('specicific', 1), ('importance.it', 1), ('24.90.230.216', 1), ('bilinguals', 1), ('retransmit', 1), ('allalalalalalalalalalalalalalallalalalalalllalalala', 1), ('politics.secondly', 1), ('176.15.89.46', 1), ('come.and', 1), ('deretic', 1), ('robj', 1), ('turkmentokelau', 1), ('perspicuity', 1), ('circcle', 1), ('mattoso', 1), ('particulare', 1), ('editorsof', 1), ('***crotalus***', 1), ('mathausen', 1), ('slow-healing', 1), ('harvards', 1), ('www.last.fm/music/melon+diesel', 1), ('removed_lines', 1), ("'chuck", 1), ('cohabiting', 1), ('atsc', 1), ('27/09/1992', 1), ('decisiv', 1), ('0-8359-1855-6', 1), ('reiss', 1), ('pianto', 1), ('hydroxylase', 1), ('secdef', 1), ('iius', 1), ('fish-derived', 1), ("isn't.keetoowah", 1), ('homin', 1), ('lifeistheretobethere', 1), ('mpen320', 1), ('evreryone', 1), ('mouth-full', 1), ('mosmof', 1), ('doxxing', 1), (':eric', 1), ('category:1970', 1), ('developed.', 1), ('otherplaces2|ha

In [50]:
OUTPUT_VOCAB = ['toxic','severe_toxic','obscene', 'threat','insult','identity_hate']
INPUT_VOCAB = ['<<PAD>>', '<<UNK>>'] + list(set(INPUT_VOCAB + OUTPUT_VOCAB))

#### builds WORD_INDEX

In [51]:
WORD_INDEX = {w: i for i, w in enumerate(INPUT_VOCAB)}
OUTPUT_WORD_INDEX = {w: i for i, w in enumerate(OUTPUT_VOCAB)}
OUTPUT_IDS = [OUTPUT_WORD_INDEX[i] for i in OUTPUT_VOCAB]

In [52]:
sorted(list(WORD_INDEX.items()), key=lambda x: x[1])[:10], WORD_INDEX['<<PAD>>'], INPUT_VOCAB[0],  INPUT_VOCAB[167046]

([('<<PAD>>', 0),
  ('<<UNK>>', 1),
  ('befoer', 2),
  ('baltimore', 3),
  ('omens', 4),
  ('jiunkrn', 5),
  ('n.g', 6),
  ('iii|age=1|units=year', 7),
  ('artic', 8),
  ('harrison.', 9)],
 0,
 '<<PAD>>',
 'isidore')

## tests INPUTVOCAB and WORD_INDEX mapping

In [53]:
import random
_i = train_datapoints[random.choice(range(len(train_datapoints)))]
print(_i.comment_text)
print("""


""")
print(
      ' '.join( [INPUT_VOCAB[i] for i in 
                 [WORD_INDEX[j] for j in word_tokenize(_i.comment_text)]]
              )
     )

"

image tagging image:centronics connector.jpg

 this media may be deleted.

thanks for uploading image:centronics connector.jpg. i notice the 'image' page currently doesn't specify who created the content, so the copyright status is therefore unclear. if you have not created this media yourself then you need to argue that we have the right to use the media on wikipedia (see copyright tagging below). if you have not created the media yourself then you should also specify where you found it, i.e., in most cases link to the website where you got it, and the terms of use for content from that page.

if the media also doesn't have a copyright tag then you must also add one. if you created/took the picture, audio, or video then you can use  to release it under the gfdl. if you believe the media qualifies as fair use, please read fair use, and then use a tag such as  or one of the other tags listed at wikipedia:image copyright tags#fair_use.  see wikipedia:image copyright tags for the full 

## Baseline model

In [54]:
class Model(nn.Module):
    def __init__(self, Config, input_vocab_size, output_vocab_size):
        super(Model, self).__init__()
        self.input_vocab_size = input_vocab_size
        self.output_vocab_size = output_vocab_size
        self.hidden_dim = Config.hidden_dim

        self.embed = nn.Embedding(self.input_vocab_size, self.hidden_dim)
        self.encode = nn.GRUCell(self.hidden_dim, self.hidden_dim)
        self.dropout = nn.Dropout(0.5)
        self.classify = nn.Linear(self.hidden_dim, self.output_vocab_size)

        self.log = model_logger.getLogger('model')
        self.log.setLevel(logging.INFO)
        if Config.cuda:
            self.cuda()
        
    def init_hidden(self, batch_size):
        ret = torch.zeros(batch_size, self.hidden_dim)
        if Config().cuda: ret = ret.cuda()
        return Variable(ret)
    
    def forward(self, seq):
        seq = Variable(torch.LongTensor(seq))
        if Config().cuda: seq = seq.cuda()
        batch_size = seq.size()[0]
        self.log.debug('{} seq size: {}'.format(type(seq.data), seq.size()))
        seq_emb = self.embed(seq).transpose(1,0)
        output = self.init_hidden(batch_size)
        for token_emb in seq_emb:
            self.log.debug('token_emb := {}'.format(token_emb))
            self.log.debug('output := {}'.format(output))
            output = self.encode(token_emb, output)
            output = self.dropout(output)
                    
        self.log.debug('output := {}'.format(output))
        ret = F.sigmoid(self.classify(output))
        self.log.debug('ret := {}'.format(ret))
        self.log.debug('ret size: {}'.format(ret.size()))

        return ret

### Batching utils

In [55]:
import numpy as np
def seq_maxlen(seqs):
    return max([len(seq) for seq in seqs])

PAD = WORD_INDEX[INPUT_VOCAB[0]]
print(PAD)
def pad_seq(seqs, maxlen=0, PAD=PAD):
    if type(seqs[0]) == type([]):
        maxlen = maxlen if maxlen else seq_maxlen(seqs)
        def pad_seq_(seq):
            return seq + [PAD]*(maxlen-len(seq))
        seqs = [ pad_seq_(seq) for seq in seqs ]
    return seqs

def batchop(datapoints, *args, **kwargs):
    indices = [d.id for d in datapoints]
    seq   = pad_seq([ [WORD_INDEX[w] for w in word_tokenize(d.comment_text)]
                     for d in datapoints])
    target = [(d.toxic, d.severe_toxic, d.obscene, d.threat, d.insult, d.identity_hate)
              for d in datapoints]
    seq, target = np.array(seq), np.array(target)
    return indices, (seq, ), (target,)

0


## Loss and accuracy function

In [85]:
def loss(output, target, loss_function=nn.MSELoss(), *args, **kwargs):
    loss = 0
    target = target[0]
    target = Variable(torch.FloatTensor(target))
    log.debug('sizes: output, target: {} {}'.format(output.size(), target.size()))
    if Config().cuda: target = target.cuda()
    return loss_function(output, target)

def accuracy(output, target, *args, **kwargs):
    target = target[0]
    target = Variable(torch.LongTensor(target))
    batch_size, class_size = target.size()    
    if Config().cuda: target = target.cuda()
    accuracy = (output > 0.5).long() != target
    return accuracy.sum().float()/batch_size/class_size
    

### repr_function to build human readable output from model

In [100]:
from IPython.display import HTML
from IPython.display import display
def repr_function(output, feed, batch_index):
    results = []
    indices, (seq,), (classes,) = feed.nth_batch(batch_index)
    for i, o, s, c in zip(indices, output, seq, classes):
        orig_s = feed.data_dict[i].comment_text
        s = [INPUT_VOCAB[i] for i in s]
        s = ' '.join(s)
        results.append([orig_s, s] + list(c))
        #o = o.max(dim=1)[1]
        results.append([' ', '  '] + [ '{:0.2f}'.format(i) for i in o.data.cpu().numpy().tolist()])
        o = o > 0.5
        results.append([' ', '  '] +  o.data.cpu().numpy().tolist())
    del indices, seq, classes
    return results

In [30]:
_train_datapoints = train_datapoints

In [77]:
import logging
log.setLevel(logging.INFO)

## Experiment on model 1

In [ ]:
import random
def experiment(epochs=1, checkpoint=1, test=False):
    model =  Model(Config(), len(INPUT_VOCAB), len(OUTPUT_VOCAB))
    if Config().cuda:  model = model.cuda()
    
    _train_datapoints = train_datapoints
    if test:
        _train_datapoints = _train_datapoints[:2000]
        print(len(_train_datapoints))
        log.setLevel(logging.INFO)
        epochs = 1
    else:
        log.setLevel(logging.INFO)

    split_index = int( len(_train_datapoints) * 0.85 )
    train_feed = DataFeed(_train_datapoints[:split_index], batchop=batchop, batch_size=128)
    test_feed = DataFeed(_train_datapoints[split_index:], batchop=batchop, batch_size=128)
    print(test_feed.num_batch)
    trainer = Trainer(model=model, loss_function=loss, accuracy_function=accuracy, 
                    checkpoint=checkpoint, epochs=epochs,
                    feeder = Feeder(train_feed, test_feed))

    predictor = Predictor(model=model, repr_function=repr_function, feed=test_feed)
    output, results = predictor.predict(random.choice(range(test_feed.num_batch)))

    for e in range(10000):
        output, results = predictor.predict(random.choice(range(test_feed.num_batch)))
        display(HTML(results._repr_html_()))
        trainer.train()
        
experiment(test=False)

2018-01-30 22:29:52,423:root:INFO   :           getLogger:: creating logger for model under MODEL

100%|██████████| 23936/23936 [00:00<00:00, 3831426.19it/s]

187


""" you should have a userbox """"this user is canadian, he is a french bastard"""" ""","`` you should have a userbox `` '' this user is canadian , he is a french bastard '' '' `` <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,1,0
,,0.55,0.63,0.57,0.42,0.66,0.50
,,1,1,1,0,1,1
"i think we should keep the current title, based on all the reasons given. (talk)","i think we should keep the current title , based on all the reasons given . ( talk ) <> <> <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0.57,0.48,0.61,0.27,0.53,0.43
,,1,0,1,0,1,0
"i appreciate your assistance, dave. i'll go take a look at your suggestions now.","i appreciate your assistance , dave . i 'll go take a look at your suggestions now . <> <> <> <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0.56,0.55,0.49,0.35,0.69,0.49
,,1,1,0,0,1,0
why'd you protect doctor steel from re-creation? there was no fair use violation.,why 'd you protect doctor steel from re-creation ? there was no fair use violation . <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.51,0.54,0.53,0.57,0.60,0.28


2018-01-30 22:29:53,044:TRAINER.main:CRITICAL:               train:: memory consumed : 7997550592

100%|██████████| 1059/1059 [09:15<00:00, 28.47s/it]
2018-01-30 22:39:09,046:TRAINER.main:INFO   :               train:: -- 0 -- loss: 0.07190301269292831
100%|██████████| 187/187 [00:15<00:00,  9.57it/s]
2018-01-30 22:39:24,523:TRAINER.main:INFO   : do_every_checkpoint:: -- 0 -- loss: 0.04056502506136894, accuracy: Variable containing:
1.00000e-02 *
  4.5573
[torch.cuda.FloatTensor of size 1 (GPU 0)]



the six feet under logo could do with a transparent background.,the six feet under logo could do with a transparent background . <> <> <> <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.08,0.03,0.04,0.04,0.04,0.05
,,0,0,0,0,0,0
no prob the image looks fine now - thanks for the heads-up! @,no prob the image looks fine now - thanks for the heads-up ! @ <> <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.03,0.08,0.06,0.01,0.03,0.03
,,0,0,0,0,0,0
"today is friday tommorow, i go to work and play with my toys.","today is friday tommorow , i go to work and play with my toys . <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0.05,0.05,0.12,0.05,0.07,0.05
,,0,0,0,0,0,0
"thank you, the dedicatee will be pleased as well, did you know?","thank you , the dedicatee will be pleased as well , did you know ? <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0.04,0.05,0.07,0.06,0.05,0.05


...

2018-01-30 22:49:03,647:TRAINER.main:CRITICAL:               train:: memory consumed : 6774906880
100%|██████████| 1059/1059 [09:13<00:00,  6.43it/s] 
2018-01-30 22:58:16,910:TRAINER.main:INFO   :               train:: -- 0 -- loss: 0.01202208548784256
100%|██████████| 187/187 [00:32<00:00,  4.95it/s]
2018-01-30 22:58:49,762:TRAINER.main:INFO   : do_every_checkpoint:: -- 0 -- loss: 0.020578917115926743, accuracy: Variable containing:
1.00000e-02 *
  2.0833
[torch.cuda.FloatTensor of size 1 (GPU 0)]



(sorry about the trolling. it just came over me after reading through the previous note. imc.),( sorry about the trolling . it just came over me after reading through the previous note . imc . ) <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.00,0.01,0.03,0.00,0.03,0.02
,,0,0,0,0,0,0
"sure, no prob. i'm glad you understand. just be careful what you label as vandalism next time.","sure , no prob . i 'm glad you understand . just be careful what you label as vandalism next time . <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0.02,0.02,0.02,0.02,0.05,0.01
,,0,0,0,0,0,0
do you know if that was incorrect introduction. how about you find me proof that it was fake?!,do you know if that was incorrect introduction . how about you find me proof that it was fake ? ! <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.03,0.03,0.03,0.03,0.03,0.02
,,0,0,0,0,0,0
"dombeya rotundifolia hi, could you tell me what has been happening at the above page? thanks","dombeya rotundifolia hi , could you tell me what has been happening at the above page ? thanks <> <> <> <> <> <> <> <> <> <> <>",0,0,0,0,0,0
,,0.00,0.01,0.03,0.01,0.02,0.01


2018-01-30 22:58:49,917:TRAINER.main:CRITICAL:               train:: memory consumed : 6774906880
100%|██████████| 1059/1059 [09:12<00:00,  4.52it/s] 
2018-01-30 23:08:02,055:TRAINER.main:INFO   :               train:: -- 0 -- loss: 0.020563490688800812
100%|██████████| 187/187 [00:47<00:00,  3.26it/s]
2018-01-30 23:08:49,634:TRAINER.main:INFO   : do_every_checkpoint:: -- 0 -- loss: 0.01712505705654621, accuracy: Variable containing:
1.00000e-02 *
  1.9531
[torch.cuda.FloatTensor of size 1 (GPU 0)]



actually you are into trouble. take care.,actually you are into trouble . take care . <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.01,0.02,0.04,0.01,0.03,0.01
,,0,0,0,0,0,0
"my pleasure. thanks for your wp:agf, too.","my pleasure . thanks for your wp : agf , too . <> <> <> <>",0,0,0,0,0,0
,,0.02,0.02,0.03,0.01,0.04,0.01
,,0,0,0,0,0,0
"do you? no? as long as you cant prove it,","do you ? no ? as long as you cant prove it , <> <> <>",0,0,0,0,0,0
,,0.04,0.04,0.06,0.05,0.12,0.01
,,0,0,0,0,0,0
force raised by the general staff special,force raised by the general staff special <> <> <> <> <> <> <> <> <>,0,0,0,0,0,0
,,0.03,0.02,0.07,0.02,0.07,0.04


2018-01-30 23:08:49,737:TRAINER.main:CRITICAL:               train:: memory consumed : 6774906880
 19%|█▉        | 201/1059 [01:41<10:39,  1.34it/s]

In [96]:
dummy_feed = DataFeed(train_datapoints[:100], batchop=batchop, batch_size=1)
indices, (seq,), (target,) = dummy_feed.nth_batch(random.choice(range(dummy_feed.num_batch)))
print(dummy_feed.data_dict[indices[0]])
print([INPUT_VOCAB[i] for i in seq[0]])

100%|██████████| 100/100 [00:00<00:00, 334474.00it/s]

Sample(id='003d77a20601cec1', comment_text="thanks much - however, if it's been resolved, why is it that today, when i didn't write anything in edit summary, camera put its ad in my edit summary box?1equalvoice1 (talk)", toxic=0, severe_toxic=0, obscene=0, threat=0, insult=0, identity_hate=0)
['thanks', 'much', '-', 'however', ',', 'if', 'it', "'s", 'been', 'resolved', ',', 'why', 'is', 'it', 'that', 'today', ',', 'when', 'i', 'did', "n't", 'write', 'anything', 'in', 'edit', 'summary', ',', 'camera', 'put', 'its', 'ad', 'in', 'my', 'edit', 'summary', 'box', '?', '1equalvoice1', '(', 'talk', ')']
